In [2]:
!pip install -U keras gensim 
!pip install -U -q PyDrive

    100% |████████████████████████████████| 307kB 5.4MB/s 
    100% |████████████████████████████████| 23.5MB 1.0MB/s 
    100% |████████████████████████████████| 51kB 10.9MB/s 
    100% |████████████████████████████████| 1.4MB 4.2MB/s 
    100% |████████████████████████████████| 133kB 20.5MB/s 
    100% |████████████████████████████████| 61kB 16.5MB/s 
    100% |████████████████████████████████| 4.7MB 3.3MB/s 
    100% |████████████████████████████████| 552kB 7.6MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /root/.cache/pip/wheels/73/f1/9b/ccf93d4ba073b6f79b1ed9df68ab5ce048d8136d0efcf90b30
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file
  Found existing installation: Keras 2.1.6
    Uninstalling Keras-2.1.6:
      Successfully uninstalled Keras-2.1.6


In [3]:
!wget http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz

--2018-09-06 14:43:55--  http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17329808 (17M) [application/x-tar]
Saving to: ‘news20.tar.gz’

news20.tar.gz       100%[===================>]  16.53M  1.72MB/s    in 9.8s    

2018-09-06 14:44:05 (1.68 MB/s) - ‘news20.tar.gz’ saved [17329808/17329808]



In [0]:
!tar xzf news20.tar.gz

In [5]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2018-09-06 14:44:11--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2018-09-06 14:44:11--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  39.5MB/s    in 18s     

2018-09-06 14:44:29 (46.3 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [6]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [7]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
import cPickle


BASE_DIR = ''
GLOVE_DIR = ''
TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.4

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}
                with open(fpath, **args) as f:
                    t = f.read()
                    i = t.find('\n\n')  # skip header
                    if 0 < i:
                        t = t[i:]
                    texts.append(t)
                labels.append(label_id)

print('Found %s texts.' % len(texts))

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


Using TensorFlow backend.


Indexing word vectors.
Found 400000 word vectors.
Processing text dataset
Found 19997 texts.
Found 174105 unique tokens.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)
Preparing embedding matrix.


In [7]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
pretrained_embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                embeddings_initializer=Constant(embedding_matrix),
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False)

print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = pretrained_embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

Training model.
Train on 11999 samples, validate on 7998 samples
Epoch 1/10
11999/11999 [==============================] - 155s 13ms/step - loss: 2.5810 - acc: 0.1744 - val_loss: 2.3600 - val_acc: 0.2256
Epoch 2/10
11999/11999 [==============================] - 154s 13ms/step - loss: 1.7368 - acc: 0.3930 - val_loss: 1.7241 - val_acc: 0.4102
Epoch 3/10
11999/11999 [==============================] - 155s 13ms/step - loss: 1.3533 - acc: 0.5262 - val_loss: 1.2453 - val_acc: 0.5618
Epoch 4/10
11999/11999 [==============================] - 154s 13ms/step - loss: 1.1175 - acc: 0.6111 - val_loss: 1.2066 - val_acc: 0.5898
Epoch 5/10
11999/11999 [==============================] - 155s 13ms/step - loss: 0.9545 - acc: 0.6771 - val_loss: 1.0347 - val_acc: 0.6577
Epoch 6/10
11999/11999 [==============================] - 154s 13ms/step - loss: 0.8164 - acc: 0.7197 - val_loss: 1.1297 - val_acc: 0.6144
Epoch 7/10
11999/11999 [==============================] - 156s 13ms/step - loss: 0.6940 - acc: 0.7638

In [38]:
random_embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                input_length=MAX_SEQUENCE_LENGTH)

print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = random_embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

Training model.
Train on 11999 samples, validate on 7998 samples
Epoch 1/10
11999/11999 [==============================] - 182s 15ms/step - loss: 2.7592 - acc: 0.0990 - val_loss: 2.3958 - val_acc: 0.1614
Epoch 2/10
11999/11999 [==============================] - 185s 15ms/step - loss: 2.2289 - acc: 0.1878 - val_loss: 2.1106 - val_acc: 0.2142
Epoch 3/10
11999/11999 [==============================] - 183s 15ms/step - loss: 1.9787 - acc: 0.2557 - val_loss: 1.9712 - val_acc: 0.2548
Epoch 4/10
11999/11999 [==============================] - 185s 15ms/step - loss: 1.6896 - acc: 0.3502 - val_loss: 1.7153 - val_acc: 0.3772
Epoch 5/10
11999/11999 [==============================] - 184s 15ms/step - loss: 1.3630 - acc: 0.4739 - val_loss: 1.5474 - val_acc: 0.4684
Epoch 6/10
11999/11999 [==============================] - 184s 15ms/step - loss: 1.0884 - acc: 0.5945 - val_loss: 1.4860 - val_acc: 0.5068
Epoch 7/10
11999/11999 [==============================] - 184s 15ms/step - loss: 0.8437 - acc: 0.6969

In [0]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [9]:
listed = drive.ListFile({'q': "'12C0S8nP6RTyF0lPbUnUA-5V0chnN3UqG' in parents"}).GetList()
for file in listed:
    print('title {}, id {}'.format(file['title'], file['id']))

title visual_keyed_vectors.cpickle, id 1NA5OuE6ZJpPwogioKYF3fSSeJiW1Nb01


In [0]:
download_path = os.path.expanduser('~/data2')
os.makedirs(download_path)

In [0]:
output_file = os.path.join(download_path, 'visual_keyed_vectors.cpickle')
temp_file = drive.CreateFile({'id': '1NA5OuE6ZJpPwogioKYF3fSSeJiW1Nb01'})
temp_file.GetContentFile(output_file)

In [0]:
visual_embeddings = cPickle.load(open(download_path + '/visual_keyed_vectors.cpickle'))

In [27]:
print(visual_embeddings.shape)

AttributeError: ignored

In [13]:
words_not_in_model = []
visual_embedding_matrix_orig = np.zeros((num_words, 2048))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    
    if word in visual_embeddings:
        visual_embedding_vector = visual_embeddings[word]
        visual_embedding_matrix_orig[i] = visual_embedding_vector
    else:
        words_not_in_model.append(word)

len(words_not_in_model)

891

In [14]:
visual_embedding_matrix_orig.shape

(20000, 2048)

In [0]:
from sklearn.decomposition import PCA

In [34]:
visual_embedding_matrix = PCA(n_components=100).fit_transform(visual_embedding_matrix_orig)
visual_embedding_matrix.shape

(20000, 100)

In [0]:
visual_embedding_layer = Embedding(num_words,
                                2048,
                                embeddings_initializer=Constant(visual_embedding_matrix_orig),
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False)

print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = visual_embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=15,
          validation_data=(x_val, y_val))

Training model.
Train on 11999 samples, validate on 7998 samples
Epoch 1/15
11999/11999 [==============================] - 55s 5ms/step - loss: 3.0930 - acc: 0.0679 - val_loss: 2.9072 - val_acc: 0.0850
Epoch 2/15
11999/11999 [==============================] - 54s 4ms/step - loss: 2.8181 - acc: 0.1128 - val_loss: 2.7333 - val_acc: 0.1383
Epoch 3/15
11999/11999 [==============================] - 54s 4ms/step - loss: 2.5617 - acc: 0.1853 - val_loss: 2.3999 - val_acc: 0.2111
Epoch 4/15
11999/11999 [==============================] - 54s 5ms/step - loss: 2.3890 - acc: 0.2325 - val_loss: 2.3110 - val_acc: 0.2246
Epoch 5/15
11999/11999 [==============================] - 54s 4ms/step - loss: 2.2564 - acc: 0.2707 - val_loss: 2.2551 - val_acc: 0.2561
Epoch 6/15
11999/11999 [==============================] - 54s 5ms/step - loss: 2.1075 - acc: 0.3124 - val_loss: 2.1118 - val_acc: 0.2953
Epoch 7/15
11999/11999 [==============================] - 54s 5ms/step - loss: 1.8984 - acc: 0.3808 - val_loss: 2